In [1]:
from numpy import genfromtxt
import csv
import numpy
import random
import pandas
import re
import time
import numpy as np

from IPython.display import Audio, display
def allDone():
    urL = 'http://www.wavsource.com/snds_2020-03-30_7102365145747638/animals/cat_meow2.wav'
    display(Audio(url=urL, autoplay=True))
allDone()

random.seed(666)

## Elongate sequences for comparison testing with DeepPhos and MusiteDeep

In [22]:
test_motifs = np.genfromtxt('data__for_nerds/new_test_motifs.csv',dtype='U')
test_motifxFamMatrix = np.genfromtxt('data__for_nerds/new_test_motifxFamMatrix.csv',delimiter=',',dtype=int)
fams = np.genfromtxt('data__for_nerds/fams.csv',dtype='U')

print(len(test_motifs))

1179


In [23]:
# f = open("prots_i_need.txt","r")
# prots_i_need = f.read().split()

In [24]:
from Bio import SeqIO

f = 'data_raw__for_nerds/uniprot_human.fasta'
fasta_content = SeqIO.parse(open(f),'fasta')

uniprot_seqs = []
prots = []

for fasta in fasta_content:
    name, sequence = fasta.id, str(fasta.seq)
    prot = name[name.find('|')+1:]
    prot = prot[:prot.find('|')]
    prots.append(prot)
    if prot == 'P02316':
        print(prot,sequence)
        break
    uniprot_seqs.append(sequence)
print(len(uniprot_seqs))

188558


In [25]:
f = 'data_raw__for_nerds/uniprot_non-human.fasta'
fasta_content = SeqIO.parse(open(f),'fasta')

for fasta in fasta_content:
    name, sequence = fasta.id, str(fasta.seq)
    prot = name[name.find('|')+1:]
    prot = prot[:prot.find('|')]
    prots.append(prot)
    uniprot_seqs.append(sequence)
print(len(set(uniprot_seqs)))

454450


In [26]:
f = 'data_raw__for_nerds/uniprot_non-human_3.fasta'
fasta_content = SeqIO.parse(open(f),'fasta')

for fasta in fasta_content:
    name, sequence = fasta.id, str(fasta.seq)
    prot = name[name.find('|')+1:]
    prot = prot[:prot.find('|')]
    prots.append(prot)
    uniprot_seqs.append(sequence)
print(len(set(uniprot_seqs)))

454453


In [27]:
uniprot_seqs = list(set(uniprot_seqs))

In [29]:
start = time.time()

motif_to_longSeq = {mot:[] for mot in test_motifs}

found = []
for i,motif in enumerate(test_motifs):
    
    if i%100==0:
        print(i)
    
    success = 0
    motif_trim = ''
    front = False
    back = False
    
    if motif[0]=='_':
        motif_trim = motif[motif.rfind('_')+1:] # reverse find
        front = True
    elif motif[-1]=='_':
        motif_trim = motif[:motif.find('_')]
        back = True
        
    for seq in uniprot_seqs:
        if front:
            if seq[:len(motif_trim)]==motif_trim:
                found.append(motif)
                motif_to_longSeq[motif].append(seq)
                success = 1
        elif back:
            if seq[-len(motif_trim):]==motif_trim:
                found.append(motif)
                motif_to_longSeq[motif].append(seq)
                success = 1
        elif motif in seq:
            found.append(motif)
            motif_to_longSeq[motif].append(seq)
            success = 1
            
    if not success:
        print(i,motif,motif_trim)
            
print("%5.3f mins" % ((time.time()-start)/60))

0
100
200
300
400
500
600
700
800
900
1000
1100
5.473 mins


In [32]:
print(len(found))
print(len(set(found)))

15207
1179


In [36]:
motif_to_randSeq = {motif:0 for motif in motif_to_longSeq.keys()}
for k,v in motif_to_longSeq.items():
    motif_to_randSeq[k] = random.choice(v)

In [210]:
def retrieve_longmer(motif,sequence,flank):

    front = False
    back = False
    
    if motif[0]=='_':
        motif_trim = motif[motif.rfind('_')+1:] # reverse find
        buff_length = motif.count('_')
        front = True
    elif motif[-1]=='_':
        motif_trim = motif[:motif.find('_')]
        back = True
        buff_length = motif.count('_')
    if front:
        phosphosite_idx = 7 - buff_length
    elif back:
        phosphosite_idx = len(sequence) - (7-buff_length) - 1
    else:
        motifStart_idx = sequence.find(motif)
        phosphosite_idx = motifStart_idx + 7
    
    n_buff = False
    c_buff = False
    
    n_idx = phosphosite_idx - flank
    if n_idx <= 0:
        n_buff = flank - phosphosite_idx
        n_idx = 0
    c_idx = phosphosite_idx + flank + 1    
    if c_idx > len(sequence)-1:
        c_buff = phosphosite_idx + flank - len(sequence) + 2
        c_idx = len(sequence)-1 
        
    motif = sequence[n_idx:c_idx]
    
    #### buffs are either * or - depending on musite vs deepphos
    if n_buff:
        motif = '*'*n_buff + motif 
    if c_buff:
        motif = motif + '*'*c_buff
        
    return motif
        

In [211]:
stuff = []
for k,v in motif_to_randSeq.items():
    stuff.append([k,retrieve_longmer(k,v,25)])
for each in stuff:
    print(each[0],len(each[1]),each[1])

In [208]:
#### FOR MUSITE DEEP MODEL ####

# ofile = open("test_musite.fasta", "w")
# for i,seq in enumerate(stuff):
#     ofile.write(">" + str(i) + "\n" + seq[1] + "\n")

# ofile.close()

In [219]:
#### FOR DEEP PHOS MODEL ####

# import pandas as pd

# data = []
# for i,each in enumerate(stuff):
#     motif = each[1]
#     motif = motif.replace('X',"*") 
#     line = [i,26,motif]
#     data.append(line)

# df = pd.DataFrame(data)
# df.to_csv('test_deepphos.csv',header=None,index=None)